In [44]:
import os
import glob
import re
import pandas as pd
import numpy as np
import PyPDF2

path = os.getcwd()
pdf_files = glob.glob(path + "/*.pdf")
print((pdf_files))

['c:\\Users\\jddel\\.vscode\\TT-Buscador-y-recomendador\\protocolo_prueba.pdf']


In [3]:
import io
from PIL import Image
from PyPDF2 import PdfReader
from PIL import ImageEnhance
from PIL import ImageFilter
import pyocr
import pyocr.builders

tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    # The tools are returned in the recommended order of 
tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))
# Ex: Will use tool 'libtesseract'
langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[2]
print("Will use lang '%s'" % (lang))

# abrir el PDF escaneado
with open('protocolo_prueba.pdf', 'rb') as file:
    # crear un objeto PdfFileReader
    pdf_reader = PdfReader(file)

    # leer la primera página del PDF y convertirla en una imagen
    page = pdf_reader.pages[0]
    text = page.extract_text()
    xObject = page['/Resources']['/XObject'].get_object()
    # Iterar sobre los objetos XObject hasta encontrar la imagen
    image = None
    for obj in xObject:
        if xObject[obj]['/Subtype'] == '/Image':
            # Obtener los datos de la imagen
            image = xObject[obj]._data
            # Crear una imagen PIL a partir de los datos
            image = Image.open(io.BytesIO(image))

            brightness = ImageEnhance.Brightness(image)
            image = brightness.enhance(1.5)
            image = image.filter(ImageFilter.SHARPEN)
            contrast = ImageEnhance.Contrast(image)
            image = contrast.enhance(1.5)
            # Mostrar la imagen
            image = image.convert('L')
            image.show()
            text += "\n" + tool.image_to_string(image, lang='spa',builder=pyocr.builders.TextBuilder())

    # imprimir el texto extraído del PDF
    print(text)


No OCR tool found


IndexError: list index out of range

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))